In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys; sys.path.insert(0, '..')

In [3]:
import osmnx as ox
import pandas as pd

from util import extract_maxspeed

In [4]:
ox.settings.useful_tags_way = ox.settings.useful_tags_way + ["cycleway"]

In [5]:
# get network for a city
G = ox.graph_from_place(
    "Somerville, Massachusetts, USA",
    network_type="bike")

# convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

In [7]:
edges["maxspeed_int"] = edges["maxspeed"].apply(extract_maxspeed).astype("Int64")

In [11]:
edges['cycleway'].unique()

TypeError: unhashable type: 'list'

In [28]:
# edges = edges.reset_index()

In [29]:
edges['maxspeed'] = max_if_list(edges['maxspeed'])

In [30]:
edges

osmid  \
u           v           key                                         
61151272    61151274    0                                 8603503   
            9588223611  0                                 8603503   
            71921695    0                               172307046   
61151274    61155754    0                                 8603503   
            61151272    0                                 8603503   
...                                                           ...   
13295850473 7307811424  0                                 9429931   
            13295850474 0    [1449558419, 1449558420, 1449558421]   
13295850474 8534819490  0                                 8605193   
            7961610380  0                                 8605193   
            13295850473 0    [1449558419, 1449558420, 1449558421]   

                                  highway lanes maxspeed            name  \
u           v           key                                                
61151272    61151274    0     residential     2   20 mph   Munroe Street   
            9588223611  0     residential     2   20 mph   Munroe Street   
            71921695    0     residential   NaN   20 mph  Bigelow Street   
61151274    61155754    0     residential     2   20 mph   Munroe Street   
            61151272    0     residential     2   20 mph   Munroe Street   
...                                   ...   ...      ...             ...   
13295850473 7307811424  0    unclassified     2   25 mph      Joy Street   
            13295850474 0         service   NaN      NaN             NaN   
13295850474 8534819490  0    unclassified     2   25 mph  Linwood Street   
            7961610380  0    unclassified     2   25 mph  Linwood Street   
            13295850473 0         service   NaN      NaN             NaN   

                            width  oneway reversed      length  \
u           v           key                                      
61151272    61151274    0    15.2   False    False   65.325668   
            9588223611  0    15.2   False     True   75.848925   
            71921695    0     NaN   False     True   76.432294   
61151274    61155754    0    15.2   False    False   88.594638   
            61151272    0    15.2   False     True   65.325668   
...                           ...     ...      ...         ...   
13295850473 7307811424  0    15.2   False     True  141.313951   
            13295850474 0     NaN   False     True   99.588979   
13295850474 8534819490  0    15.2   False     True  105.556862   
            7961610380  0    15.2   False    False   63.683190   
            13295850473 0     NaN   False    False   99.588979   

                                                                      geometry  \
u           v           key                                                      
61151272    61151274    0    LINESTRING (-71.0939 42.38221, -71.09354 42.38...   
            9588223611  0    LINESTRING (-71.0939 42.38221, -71.09396 42.38...   
            71921695    0    LINESTRING (-71.0939 42.38221, -71.09387 42.38...   
61151274    61155754    0    LINESTRING (-71.09332 42.38181, -71.09324 42.3...   
            61151272    0    LINESTRING (-71.09332 42.38181, -71.0934 42.38...   
...                                                                        ...   
13295850473 7307811424  0    LINESTRING (-71.08755 42.37956, -71.08745 42.3...   
            13295850474 0    LINESTRING (-71.08755 42.37956, -71.08762 42.3...   
13295850474 8534819490  0    LINESTRING (-71.08852 42.37902, -71.08779 42.3...   
            7961610380  0    LINESTRING (-71.08852 42.37902, -71.08892 42.3...   
            13295850473 0    LINESTRING (-71.08852 42.37902, -71.08844 42.3...   

                            cycleway  ref service access bridge  \
u           v           key                                       
61151272    61151274    0        NaN  NaN     NaN    NaN    NaN   
            9588223611  0        NaN  NaN     

In [11]:
edges['cycleway'] = first_if_list(edges['cycleway'])

In [ ]:
max_if_list

In [21]:
edges['cycleway'].unique()

array([nan, 'lane', 'shared_lane', 'no', 'separate', 'track', 'crossing',
       'share_busway', 'shared', 'construction', 'link'], dtype=object)

In [14]:
highways = edges.reset_index()['highway']

In [24]:
unique = set()
for _, v in highways.items():
    if isinstance(v, list):
        unique.update(v)
    else:
        unique.add(v)
unique

{'busway',
 'cycleway',
 'living_street',
 'path',
 'pedestrian',
 'primary',
 'primary_link',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'tertiary',
 'trunk',
 'trunk_link',
 'unclassified'}

In [ ]:
import numpy as np
import re

In [4]:
edges[['highway', 'name', 'maxspeed']].sample(10)

,,,highway,name,maxspeed
u,v,key,,,
7983674950,71916827,0,residential,Belmont Street,20 mph
11777326029,11777326029,1,service,NaN,NaN
61172626,61174071,0,trunk,Fellsway,NaN
1949661741,1155703191,0,residential,Latin Way,NaN
8334781996,8334781999,0,service,NaN,NaN
61170625,61170638,0,secondary,Summer Street,20 mph
61170914,71917815,0,secondary,Summer Street,20 mph
71914380,71927580,0,residential,Edgar Avenue,20 mph
71947029,71921397,0,secondary,Grand Union Boulevard,20 mph


In [7]:
def extract_maxspeed(x):
    # Handle NaN early
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan

    # Handle lists and numpy arrays uniformly
    if isinstance(x, (list, np.ndarray)):
        speeds = [extract_maxspeed(i) for i in x]
        # filter out NaNs
        speeds = [s for s in speeds if not pd.isna(s)]
        return max(speeds) if speeds else np.nan

    # Convert to string and extract digits
    x_str = str(x)
    nums = re.findall(r'\d+', x_str)

    return int(max(nums, key=int)) if nums else np.nan

In [8]:
edges['maxspeed_int'] = edges['maxspeed'].apply(extract_maxspeed)

In [9]:
edges[['highway', 'name', 'maxspeed', 'maxspeed_int']].sample(10)

,,,highway,name,maxspeed,maxspeed_int
u,v,key,,,,
71951018,71928807,0,residential,Stickney Avenue,20 mph,20.0
61173051,71938041,0,residential,Dickson Street,NaN,NaN
71916774,71953040,0,residential,Victoria Street,NaN,NaN
71954205,945798791,0,primary,Somerville Avenue,25 mph,25.0
61182360,61178893,0,secondary,Medford Street,25 mph,25.0
61177298,11395248465,0,residential,Central Road,25 mph,25.0
604063583,71936370,0,residential,Jaques Street,20 mph,20.0
71920826,71947231,0,secondary,Broadway,25 mph,25.0
71919864,71949322,0,residential,Century Street,20 mph,20.0


In [11]:
edges.to_file("../data/somerville_roads.gpkg", layer="roads", driver="GPKG")

## Bike

In [ ]:
# Example: get driving network for a city
G = ox.graph_from_place("Somerville, Massachusetts, USA", network_type="bike")

In [ ]:
# Convert to GeoDataFrames (nodes + edges)
nodes, edges = ox.graph_to_gdfs(G)

edges_drive.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")
edges_bike.to_file("../data/somerville_bike.gpkg", layer="roads", driver="GPKG")